In [1]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications import MobileNetV2 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D 
from tensorflow.keras.models import Model 
from tensorflow.keras.optimizers import Adam 
import numpy as np
# 데이터셋 디렉토리 설정
train_dir = 'dogs-vs-cats/small_train' # O: 'data/train'
#ImageDataGenerator를 사용하여 데이터 로드 및 전처리
train_datagen = ImageDataGenerator (
    rescale=1.0/255.0, # 이미지 픽셀 값을 0-1 범위로 스케일링
    shear_range=0.2, # 랜덤한 전단 변환 적용 
    zoom_range=0.2, # 랜덤한 확대/축소 변환 적용 
    horizontal_flip=True, # 랜덤하게 이미지를 좌우. 반전 
    validation_split=0.2 # 80% 학습, 20% 검증 데이터로 분리
)
# 학습 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224, 224),# 모든 이미지 크기를 224x224로 조정
    batch_size=32, # 배치 | 크기
    class_mode='binary', # 이진 분류를 위해 라벨을 이진 형식으로 설정
    subset='training' # 학습 데이터 서브셋
)


Found 32 images belonging to 2 classes.


In [2]:

# 검증 데이터 생성기
validation_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224, 224),# 모든 이미지 크기를 224x224로 조정
    batch_size=32, # 배치크기
    class_mode='binary',# 이진 분류를 위해 라벨을 이진 형식으로 설정
    subset='validation' # 검증 데이터 서브셋
)

# MobiLeNetv2 모델 불러오기 (사전 학습된 가중치 사용, 최상위 레이어 제거)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# 모델 구성
x = base_model.output
# 특성 맵의 크기를 (None, 7, 7, 2048)에서 (None, 2048)로 변환
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
# 최종 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)
# 기본 모델의 레이어를 고정 (학습되지 않도록 설정)
for layer in base_model. layers:
    layer.trainable = False

Found 8 images belonging to 2 classes.


In [3]:
# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
# 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10 # 에포크 수
)


Epoch 1/10


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3438 - loss: 0.9687 - val_accuracy: 0.3750 - val_loss: 0.7787
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.8207
Epoch 3/10


2024-07-30 10:19:05.263483: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step - accuracy: 0.4375 - loss: 0.7528 - val_accuracy: 0.6250 - val_loss: 0.6884
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.6242
Epoch 5/10


2024-07-30 10:19:05.847883: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step - accuracy: 0.6562 - loss: 0.6075 - val_accuracy: 0.6250 - val_loss: 0.5527
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6250 - val_loss: 0.5347
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.9375 - loss: 0.4974 - val_accuracy: 0.7500 - val_loss: 0.4724
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.4317
Epoch 9/10


2024-07-30 10:19:06.947123: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step - accuracy: 1.0000 - loss: 0.3864 - val_accuracy: 0.8750 - val_loss: 0.4171
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.3864


In [4]:
# 모델 저장
model.save('cat_dog_classifier.keras')